In [6]:
import os
from pyvis.network import Network
from owlready2 import get_ontology

def load_ontology(owl_file_path):
    # Load the ontology
    onto = get_ontology(f"file://{os.path.abspath(owl_file_path)}").load()
    return onto

def create_interactive_graph(ontology):
    net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    
    # Add classes to the graph
    for cls in ontology.classes():
        net.add_node(cls.name, label=cls.name, title=cls.iri, color='skyblue')
    
    # Add object properties to the graph
    for prop in ontology.object_properties():
        for domain in prop.domain:
            for range in prop.range:
                net.add_edge(domain.name, range.name, title=prop.name, color='gray')

    return net

owl_file_path = '/home/user/pel_ws/src/pel_ros/orka/owl/orka-core.owl'
ontology = load_ontology(owl_file_path)
net = create_interactive_graph(ontology)
net.show("knowledge_graph.html", notebook=False)


knowledge_graph.html


In [2]:
print(os.environ)

environ({'CHROME_DESKTOP': 'code-url-handler.desktop', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'DESKTOP_SESSION': 'pop', 'DISPLAY': ':1', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'GDK_BACKEND': 'x11', 'GDMSESSION': 'pop', 'GIO_LAUNCHED_DESKTOP_FILE_PID': '4021839', 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated', 'GNOME_SHELL_SESSION_MODE': 'pop', 'GPG_AGENT_INFO': '/run/user/1000/gnupg/S.gpg-agent:0:1', 'GTK_MODULES': 'gail:atk-bridge:appmenu-gtk-module', 'HOME': '/home/user', 'IM_CONFIG_PHASE': '1', 'INVOCATION_ID': 'f89c68cd185a433181765c58b4b0a105', 'JOURNAL_STREAM': '8:92686', 'LANG': 'en_GB.UTF-8', 'LANGUAGE': 'en_GB:en', 'LC_ADDRESS': 'en_GB.UTF-8', 'LC_IDENTIFICATION': 'en_GB.UTF-8', 'LC_MEASUREMENT': 'en_GB.UTF-8', 'LC_MONETARY': 'en_GB.UTF-8', 'LC_NAME': 'en_GB.UTF-8', 'LC_NUMERIC': 'en_GB.UTF-8', 'LC_PAPER': 'en_GB.UTF-8', 'LC_TELEPHONE': 'en_GB.UTF-8', 'LC_TIME': 'en_GB.UTF-8', 'LOGNAME': 'user', 'MANAGERPID': '8678', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'pop:GNOM

In [3]:
import pyvis.network as net
print(net.__file__)

/home/user/pel_ws/.venv/lib/python3.8/site-packages/pyvis/network.py


In [2]:
from rdflib.namespace import SOSA, SSN
from rdflib import Graph

g = Graph()
g.bind("sosa", SOSA)  # bind an RDFLib-provided namespace to a prefix
g.bind("ssn", SSN)      # bind a user-declared namespace to a prefix



In [10]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_top_k_subclasses_by_links(entity_id, k):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    query = f"""
    SELECT ?subclass ?subclassLabel (COUNT(?item) AS ?count) WHERE {{
      ?item wdt:P279 ?subclass.
      ?subclass wdt:P279* wd:{entity_id}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} GROUP BY ?subclass ?subclassLabel ORDER BY DESC(?count) LIMIT {k}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    subclasses = []
    for result in results["results"]["bindings"]:
        subclasses.append({
            "subclass": result["subclass"]["value"],
            "subclassLabel": result["subclassLabel"]["value"],
            "count": int(result["count"]["value"])
        })
    
    return subclasses

# Example usage
entity_id = "Q13191"  # Example entity ID for "animal"
k = 10
top_k_subclasses = get_top_k_subclasses_by_links(entity_id, k)
for subclass in top_k_subclasses:
    print(f"Subclass: {subclass['subclassLabel']} (ID: {subclass['subclass']}), Count: {subclass['count']}")


Subclass: orange (ID: http://www.wikidata.org/entity/Q13191), Count: 12
Subclass: blood orange (ID: http://www.wikidata.org/entity/Q303669), Count: 1
Subclass: Navel orange (ID: http://www.wikidata.org/entity/Q3621268), Count: 1


In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_top_k_subclasses_by_outgoing_links(entity_id, k):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    query = f"""
    SELECT ?subclass ?subclassLabel (COUNT(?link) AS ?count) WHERE {{
      ?subclass wdt:P279* wd:{entity_id}.
      ?subclass wdt:P279 ?link.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} GROUP BY ?subclass ?subclassLabel ORDER BY DESC(?count) LIMIT {k}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    subclasses = []
    for result in results["results"]["bindings"]:
        subclasses.append({
            "subclass": result["subclass"]["value"],
            "subclassLabel": result["subclassLabel"]["value"],
            "count": int(result["count"]["value"])
        })
    
    return subclasses

# Example usage
entity_id = "Q13191" 
k = 10
top_k_subclasses = get_top_k_subclasses_by_outgoing_links(entity_id, k)
for subclass in top_k_subclasses:
    print(f"Subclass: {subclass['subclassLabel']} (ID: {subclass['subclass']}), Count: {subclass['count']}")


Subclass: orange (ID: http://www.wikidata.org/entity/Q13191), Count: 4
Subclass: blood orange (ID: http://www.wikidata.org/entity/Q303669), Count: 1
Subclass: arancia del Gargano (ID: http://www.wikidata.org/entity/Q3621265), Count: 1
Subclass: Arancia di Ribera (ID: http://www.wikidata.org/entity/Q3621267), Count: 1
Subclass: Navel orange (ID: http://www.wikidata.org/entity/Q3621268), Count: 1
Subclass: Larinġ t'Għawdex (ID: http://www.wikidata.org/entity/Q6703217), Count: 1
Subclass: Q10379668 (ID: http://www.wikidata.org/entity/Q10379668), Count: 1
Subclass: Q11121992 (ID: http://www.wikidata.org/entity/Q11121992), Count: 1
Subclass: Maltaise (ID: http://www.wikidata.org/entity/Q16668217), Count: 1
Subclass: Salustiana (ID: http://www.wikidata.org/entity/Q21086177), Count: 1
Subclass: Midknight Valencia orange (ID: http://www.wikidata.org/entity/Q30635271), Count: 1
Subclass: red orange of Sicily (ID: http://www.wikidata.org/entity/Q33126598), Count: 1
Subclass: Chocolate navel oran

In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_top_k_subclasses_by_incoming_links(entity_id, k):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    query = f"""
    SELECT ?subclass ?subclassLabel (COUNT(?link) AS ?count) WHERE {{
      ?subclass wdt:P279* wd:{entity_id}.
      ?link ?anylink ?subclass.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} GROUP BY ?subclass ?subclassLabel ORDER BY DESC(?count) LIMIT {k}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    subclasses = []
    for result in results["results"]["bindings"]:
        subclasses.append({
            "subclass": result["subclass"]["value"],
            "subclassLabel": result["subclassLabel"]["value"],
            "count": int(result["count"]["value"])
        })
    
    return subclasses

# Example usage
entity_id = "Q13191"  # Example entity ID for "animal"
k = 10
top_k_subclasses = get_top_k_subclasses_by_incoming_links(entity_id, k)
for subclass in top_k_subclasses:
    print(f"Subclass: {subclass['subclassLabel']} (ID: {subclass['subclass']}), Count: {subclass['count']}")


Subclass: orange (ID: http://www.wikidata.org/entity/Q13191), Count: 12
Subclass: blood orange (ID: http://www.wikidata.org/entity/Q303669), Count: 1
Subclass: Navel orange (ID: http://www.wikidata.org/entity/Q3621268), Count: 1
